In [1]:
import pandas as pd
import numpy as np

In [8]:
dataset = pd.read_csv('train.csv')
dataset.head()

,User,Track,Rating
0,1,0,27
1,4,0,10
2,8,0,30
3,10,0,30
4,11,0,14


In [9]:
dataset.shape

(150999, 3)

In [10]:
len(dataset.User.unique()), len(dataset.Track.unique())

(47856, 184)

In [12]:
dataset.User = dataset.User.astype('category').cat.codes.values
dataset.Track = dataset.Track.astype('category').cat.codes.values
dataset.head()

,User,Track,Rating
0,1,0,27
1,3,0,10
2,7,0,30
3,9,0,30
4,10,0,14


In [13]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(dataset, test_size=0.2)

In [81]:
import keras
from IPython.display import SVG
from keras.optimizers import Adam
from keras.utils.vis_utils import model_to_dot
from keras.constraints import non_neg

n_users, n_tracks = len(dataset.User.unique()), len(dataset.Track.unique())

In [100]:
n_latent_factors_user = 5
n_latent_factors_track = 8

track_input = keras.layers.Input(shape=[1],name='Item')
track_embedding = keras.layers.Embedding(n_movies + 1, n_latent_factors_track, name='Track-Embedding')(track_input)
track_vec = keras.layers.Flatten(name='FlattenMovies')(track_embedding)
track_vec = keras.layers.Dropout(0.2)(track_vec)


user_input = keras.layers.Input(shape=[1],name='User')
user_vec = keras.layers.Flatten(name='FlattenUsers')(keras.layers.Embedding(n_users + 1, n_latent_factors_user,name='User-Embedding')(user_input))
user_vec = keras.layers.Dropout(0.2)(user_vec)

In [138]:
def _concat(vectors):
    return keras.layers.concatenate(vectors, axis=-1);

In [139]:
concat = keras.layers.Lambda(_concat)([user_vec, track_vec]);

TypeError: Expected int32, got list containing Tensors of type '_Message' instead.

In [101]:
concat_dropout = keras.layers.Dropout(0.2)(concat)
dense = keras.layers.Dense(200,name='FullyConnected')(concat)
dropout_1 = keras.layers.Dropout(0.2,name='Dropout')(dense)
dense_2 = keras.layers.Dense(100,name='FullyConnected-1')(concat)
dropout_2 = keras.layers.Dropout(0.2,name='Dropout')(dense_2)
dense_3 = keras.layers.Dense(50,name='FullyConnected-2')(dense_2)
dropout_3 = keras.layers.Dropout(0.2,name='Dropout')(dense_3)
dense_4 = keras.layers.Dense(20,name='FullyConnected-3', activation='relu')(dense_3)


result = keras.layers.Dense(1, activation='relu',name='Activation')(dense_4)

NameError: name 'concat' is not defined

In [97]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
User (InputLayer)               (None, 1)            0                                            
__________________________________________________________________________________________________
Track (InputLayer)              (None, 1)            0                                            
__________________________________________________________________________________________________
NonNegUser-Embedding (Embedding (None, 1, 3)         143571      User[0][0]                       
__________________________________________________________________________________________________
NonNegMovie-Embedding (Embeddin (None, 1, 3)         555         Track[0][0]                      
__________________________________________________________________________________________________
FlattenUse

In [98]:
history = model.fit([train.User, train.Track], train.Rating, epochs=100, verbose=0)

KeyboardInterrupt: 

In [ ]:
track_embedding_learnt = model.get_layer(name='NonNegTrack-Embedding').get_weights()[0]
pd.DataFrame(track_embedding_learnt).describe()